In [9]:
!pip install selenium
!pip install PyPDF2

     ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
     - -------------------------------------- 10.2/232.6 kB ? eta -:--:--
     ---- -------------------------------- 30.7/232.6 kB 435.7 kB/s eta 0:00:01
     ----------- ------------------------- 71.7/232.6 kB 563.7 kB/s eta 0:00:01
     --------------------------------- ---- 204.8/232.6 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 232.6/232.6 kB 1.1 MB/s eta 0:00:00


In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from PyPDF2 import PdfReader
from pathlib import Path
import time

In [6]:
options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    "download.default_directory": "C:\\Users\\linag\\Science\\pdfs",
    "plugins.always_open_pdf_externally": True
})

with webdriver.Chrome(options=options) as driver:
    driver.get('https://elibrary.ru/')
    #Регистрация на elibrary
    login = driver.find_element(By.ID, 'login')
    password = driver.find_element(By.ID, 'password')
    login.send_keys('khrustitskayaSK')
    password.send_keys('Metalorka12HP&')
    time.sleep(2)
    button = driver.find_element(By.CLASS_NAME, 'butred')
    ActionChains(driver).move_to_element(button).click(button).perform()
    time.sleep(2)
    
    #Поиск по сайту
    search = driver.find_element(By.ID, 'ftext')
    search.send_keys('Машинное обучение')
    button = driver.find_element(By.CLASS_NAME, 'butblue')
    ActionChains(driver).move_to_element(button).click(button).perform()
    time.sleep(5)
    
    #Скачивание 100 научных публикаций в pdf
    count_page = 1
    count_publ = 0
    inf_publ = []
    while count_publ < 100:
        time.sleep(5)
        table_center = driver.find_element(By.CSS_SELECTOR, '#restab')
        publications = table_center.find_elements(By.TAG_NAME, 'tr')[1:]
        for publication in publications:
            ActionChains(driver).move_to_element(publication).perform()
            try:
                img = publication.find_element(By.CSS_SELECTOR, 'a img')
                if 'green' in img.get_attribute('src'):
                    count_publ += 1
                    ActionChains(driver).move_to_element(img).click().perform()
                    time.sleep(5)
            except NoSuchElementException:
                continue
        count_page += 1
        menus_page = [element for element in driver.find_elements(By.CSS_SELECTOR, '.menus td') if element.text == str(count_page)][0]
        ActionChains(driver).move_to_element(menus_page).click().perform()
    

In [33]:
# pdf2txt

pdfs = Path(Path.cwd(), 'pdfs')

res_text = []

for pdf_file in pdfs.rglob('*'):
    pdf_file = open(pdf_file, 'rb')
    pdf_reader = PdfReader(pdf_file)
    text = ''
    for page_num in range(len(pdf_reader.pages)):
        text += pdf_reader.pages[page_num].extract_text()
    res_text += [text]

XRef object at 2614 can not be read, some object may be missing


In [34]:
print(res_text[:2])

['Имя\n:\nПароль\n:\nВход\nЗабыли\n \nпароль\n?', 'Обзор алгоритмов машинного обучения,  \nрешающих задачу обнаружения спама  \nСкляренко Н.С. , МГТУ им. Н.Э. Баумана  \nnikolay .skliarenko @gmail .com \n \nАннотация  \nДанная работа посвящена решению задачи \nфильтрации спама методами машинного \nобучения. Проведён обзор алгоритмов \nидентификации спама, предложена их \nклассификация. Дано описание \nматемат ического аппарата рассмотренных \nалгори тмов, приведены данные о точности \nсущес твующих их реализаций. Выделены \nперспективные направления исследований \nс целью повышения качества \nидентификации спама.  \n1 Введение  \nСпам – массово рассылаемые сообщения \n(как напрямую, так и косвенно), \nпредназн аченные лицам, не выражавшим \nжелания их получать, несмо тря на \nпредпринятые меры по предотвращению этой \nрассылки [ Cormack , 2008]. Борьба со спамом \nидет уже не один десяток лет, и несмотря на \nуспехи в разрабо тке и исследовании \nразличных подходов к р ешению данной \n